# Ejemplo de uso de la Libreria

# Limpia los datos

## Consideraciones antes de usar el código
Para limpiar los datos, primero asegúrate de tener los siguientes archivos descargados:

1. BACI CEPII: Datos de comercio internacional
   - Ubicación: `data/raw_data/BACI_HS92_V202401b/`
   - Archivos: `BACI_HS92_Y{year}_V202401b.csv` para cada año
   - Archivo de códigos de país: `country_codes_V202401b.csv`

2. World Bank Data:
   - Ubicación: `data/raw_data/world_bank_data/`
   - Archivo de países: `countries.csv`
   - Archivo de deflactor del PIB: `NY.GDP.DEFL.ZS.AD_1995-2023.csv`

Asegúrate de que estos archivos estén en las ubicaciones correctas antes de ejecutar el código de limpieza.

## Cómo se limpian los datos
El proceso de limpieza de datos realiza las siguientes operaciones:

1. Carga los datos brutos de comercio, códigos de país, datos de región y deflactor del PIB.
2. Enriquece los datos de país con información de región.
3. Limpia y transforma los datos de transacciones comerciales:
   - Reemplaza valores faltantes en la columna de cantidad.
   - Mapea los códigos de país a códigos ISO de 3 letras.
   - Renombra las columnas para mayor claridad.
4. Convierte los valores monetarios a dólares constantes utilizando el deflactor del PIB.
5. Guarda los datos limpios en archivos CSV separados por año.

# Ejemplo Calcula la diversidad de alguna región


## Cálculo de la diversidad

El cálculo de la diversidad en el comercio internacional nos permite entender qué tan variada es la canasta de productos que exporta o importa una región o país. Utilizamos el índice de diversidad basado en la entropía de Shannon, que se calcula de la siguiente manera:

1. Calculamos las probabilidades marginales para cada producto:
   $P(i) =$ (Valor exportado del producto i) / (Valor total de exportaciones)

2. Calculamos la entropía de Shannon:
   $H = -∑(P(i) * log2(P(i)))$

3. Convertimos la entropía a un índice de diversidad:
   $Diversidad = 2^H$

Este índice nos da una medida de la diversidad efectiva de productos. Un valor más alto indica una mayor diversidad en la canasta de productos comerciados.


## Clasifica a los paises de acuerdo a un esquema

El esquema de clasificación nos permite agrupar países según diferentes criterios, como región geográfica, nivel de ingresos, etc. Esto facilita el análisis de patrones comerciales a nivel de grupos de países.

Para crear un esquema sin clasificación (es decir, donde cada país se clasifica como sí mismo), puedes usar el siguiente código:

```python
no_classification = ClassificationScheme(
    name="by_country"
)
```
Este esquema simplemente mapeará cada país a sí mismo, lo que es útil cuando quieres analizar países individualmente.


Para clasificaciones con el archivo csv, la clase se inicializa con un nombre y la ruta al archivo que contiene la información de clasificación. Se deben r como __key_column__ aquella donde esté el código iso de 3 dígitos y de __value_column__ aquella donde esté la clasificación.

In [1]:
from complex_trade_flow import ClassificationScheme


region_scheme = ClassificationScheme(
    name="by_region",
    file_path="data/raw_data/world_bank_data/countries.csv",
    key_column="id",
    value_column="region.value"
)    

ModuleNotFoundError: No module named 'trade_network'

## Crear la red de comercio para un año específico
Nota: Para hacer uso de TradeNetwork, asegúrate de que los archivos de datos limpios estén en la ruta correcta (generalmente en 'data/processed_data/BACI_HS92_V202401b/cleaned_trade_data/')
y que el archivo de configuración 'data_paths.json' esté correctamente configurado con las rutas correspondientes y datos.

In [2]:
from complex_trade_flow import TradeNetwork

trade_network = TradeNetwork.from_year(
    year=2020,
    classification_schemes=[region_scheme]
)


Veamos qué regiones están disponibles en nuestro esquema de clasificación:

In [3]:
trade_network.entities[str(region_scheme)]

{'East Asia & Pacific',
 'Europe & Central Asia',
 'Latin America & Caribbean ',
 'Middle East & North Africa',
 'North America',
 'South Asia',
 'Sub-Saharan Africa ',
 'Unknown'}

## Cálculo de la diversidad de exportación para una región
Ahora, calculemos la diversidad de exportación para la región 'South Asia':

In [4]:
from complex_trade_flow import DiversityCalculator

data = trade_network.filter_data_by_entities(
            scheme_name=str(region_scheme),
            exporters=['South Asia']
        )

export_diversity = DiversityCalculator.calculate_diversity_index(
                data=data
            )
print(f"Export product diversity for {'South Asia'}: {export_diversity:.2f}")

Export product diversity for South Asia: 458.12


# Analisis de Flujos de diversidad

Para facilitar el analisis de los datasets se cuentan con analisis ya programados, la función run_analysis coordina todo el proceso de análisis de diversidad. Para cada año y esquema de clasificación proporcionado, ejecuta un cálculo específico de diversificación económica mediante la función compute_entity_product_diversification, que mide cuán diversificados están los productos que un país exporta e importa. Esta última función filtra los datos comerciales por país y esquema, y calcula un índice de diversidad para las exportaciones e importaciones,

In [5]:
from complex_trade_flow import EconomicComplexityAnalyzer
from complex_trade_flow.constants import EconomicComplexity

analyzer = EconomicComplexityAnalyzer(
    start_year=1995,
    end_year=1996,
    classification_schemes=[region_scheme]
)

analyzer.run_analysis(
    type_analysis=EconomicComplexity.ENTITY_PRODUCT_DIVERSIFICATION,
    output_directory=f"data/processed_data/BACI_HS92_V202401b/{str(region_scheme)}/diversity/"
)


Analyzing by_region for 1995...


Analyzing year 1995: 100%|██████████| 8/8 [00:24<00:00,  3.04s/it]


KeyboardInterrupt: 